<a href="https://colab.research.google.com/github/IsaacFigNewton/CSC-480/blob/main/Revised_Roomba_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config

In [1]:
import networkx as nx
import json
import random
import numpy as np

contents = """7
5
_#a____
___#__#
*__#_*_
_______
____*__
"""

# save an offset map
offset_map = {
    # idx 0 => row offset
    # idx 1 => col offset
    "N": (-1, 0),
    "S": (1, 0),
    "E": (0, 1),
    "W": (0, -1)
}

## Helpers

## GameTree Class

In [2]:
class World:
  def __init__(self, file_contents:str):
    world_lst = file_contents.split("\n")
    self.num_cols = int(world_lst[0])
    self.num_rows = int(world_lst[1])
    self.grid = [[c for c in row] for row in world_lst[2:-1]]
    self.dirt_count = np.sum([np.sum([1 for c in row if c == "*"]) for row in self.grid])

    print("Successfully loaded world:")
    for row in self.grid:
      print(row)


# ensure the move is not OOB and not blocked by an obstacle
def is_legal_move(world, current_pos, proposed_pos):
  # check adjacency
  if not (proposed_pos[0] - current_pos[0], proposed_pos[1] - current_pos[1]) in offset_map.values():
    return False
  # check if the row idx is OOB
  if proposed_pos[0] < 0 or proposed_pos[0] >= world.num_rows:
    return False
  # check if the row idx is OOB
  if proposed_pos[1] < 0 or proposed_pos[1] >= world.num_cols:
    return False
  # check for obstacles
  if world.grid[proposed_pos[0]][proposed_pos[1]] == "#":
    return False
  return True

In [7]:
class Agent:
  def __init__(self, world: World):
    # vars for tracking position within game tree for DFS and UCS
    self.world = world
    self.stack = list()
    self.path = list()
    self.pos = self.get_bot_pos_from_grid()
    self.model = GameTreeNode(self.pos)


  def get_bot_pos_from_grid(self):
    for row in range(self.world.num_rows):
      for col in range(self.world.num_cols):
        if self.world.grid[row][col] == "a":
          return (row, col)
    raise ValueError("No bot found in the grid")



  def select_action(self):
      # if the bot is on dirt, clean it
      if self.world.grid[self.pos[0]][self.pos[1]] == "*":
          print("V")
          self.world.dirt_count -= 1
          self.world.grid[self.pos[0]][self.pos[1]] = "_"
          return

      # add the current node to the path
      self.path.append(self.pos)

      # expand all the current node's children
      self.model.expand_children(self.world)
      for child in self.model.children:
        # add new, unexplored children to the stack
        if child[1].pos not in self.path:
          self.stack.append(child)

      # take the first available new move
      next_move = self.stack.pop()
      print(next_move[0], next_move[1].pos)
      self.pos = next_move[1].pos
      self.model = next_move[1]

In [8]:
class GameTreeNode:
  def __init__(self, pos):
    self.pos = pos
    # a set of child game states of the form:
    #   (<edge type (N, S, E, W)>, <new GameTreeNode>)
    self.children = set()


  def __hash__(self):
    return hash(self.pos)


  def get_proposed_move_pos(self, move):
    diff = offset_map[move]
    return (
        self.pos[0] + diff[0],
        self.pos[1] + diff[1]
    )


  def expand_children(self, world: World):
    # create a list of possible moves
    possible_moves = {
        move: self.get_proposed_move_pos(move)
        for move in offset_map.keys()
    }
    # filter it down to a list of legal moves
    for move, new_pos in possible_moves.items():
      if is_legal_move(world, self.pos, new_pos):
        self.children.add((move, GameTreeNode(new_pos)))


  def prune_child(self, pos):
    del self.children[pos]

# Test DFS

In [12]:
world = World(contents)
agent = Agent(world)

Successfully loaded world:
['_', '#', 'a', '_', '_', '_', '_']
['_', '_', '_', '#', '_', '_', '#']
['*', '_', '_', '#', '_', '*', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '*', '_', '_']


In [13]:
i = 100
while world.dirt_count > 0 and i > 0:
  agent.select_action()
  print([node[1].pos for node in agent.stack])
  print(agent.world.dirt_count)
  i -= 1

E (0, 3)
[(1, 2)]
3.0
E (0, 4)
[(1, 2)]
3.0
S (1, 4)
[(1, 2), (0, 5)]
3.0
E (1, 5)
[(1, 2), (0, 5), (2, 4)]
3.0
N (0, 5)
[(1, 2), (0, 5), (2, 4), (2, 5)]
3.0
E (0, 6)
[(1, 2), (0, 5), (2, 4), (2, 5)]
3.0
S (2, 5)
[(1, 2), (0, 5), (2, 4)]
3.0
V
[(1, 2), (0, 5), (2, 4)]
2.0
E (2, 6)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5)]
2.0
S (3, 6)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5)]
2.0
S (4, 6)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5)]
2.0
W (4, 5)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5)]
2.0
N (3, 5)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5), (4, 4)]
2.0
W (3, 4)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5), (4, 4)]
2.0
N (2, 4)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5), (4, 4), (3, 3), (4, 4)]
2.0
S (4, 4)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5), (4, 4), (3, 3)]
2.0
V
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5), (4, 4), (3, 3)]
1.0
W (4, 3)
[(1, 2), (0, 5), (2, 4), (2, 4), (3, 5), (3, 5), (4, 4), (3, 3)]
1.0
W (4, 2)
[(1, 2), (0, 5), (2, 4), (2, 4), 